# 학습데이터, 테스트데이터 만들기

In [22]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [23]:
train_ft = pd.read_excel('train_tmp.xlsx')
test_ft = pd.read_excel('test_tmp.xlsx')

train_ft.shape,test_ft.shape

((5754, 8), (1439, 8))

In [24]:
train_ft = pd.concat([train_ft,test_ft])

In [25]:
train_quality = train_ft.loc[:,['리뷰','퀄리티_긍부정']].copy()
train_quality = train_quality[train_quality['퀄리티_긍부정'].isin([-1,1])]

train_quality.shape

(2298, 2)

In [26]:
train_quality['퀄리티_긍부정'].value_counts() 
# -1의 비율이 훨씬 더 높은 수준임을 알 수 있음 

퀄리티_긍부정
-1    1337
 1     961
Name: count, dtype: int64

In [27]:
train_quality.columns

Index(['리뷰', '퀄리티_긍부정'], dtype='object')

In [28]:
train_arr = train_quality['리뷰'].to_numpy()

train_arr.shape

(2298,)

In [29]:
target = train_quality['퀄리티_긍부정'].to_numpy().reshape(-1,1)
target = np.where(target == -1,0,target)
target = target.astype('int64')
target.shape

(2298, 1)

In [30]:
target.mean()

0.4181897302001741

# 사전학습 모델 선정

In [31]:
model_name = 'kykim/bert-kor-base'

In [32]:
from transformers import AutoTokenizer, AutoModel

In [33]:
model = AutoModel.from_pretrained(model_name)

/home/lab01/.local/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [34]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 사전학습모델을 바탕으로 학습 실행

In [35]:
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, x, y=None):
        self.tokenizer = tokenizer
        self.x = x
        self.y = y
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        item = {}
        item["x"] = self.get_tokenizer(self.x[idx])
        if self.y is not None:
            item["y"] = torch.Tensor(self.y[idx])
        return item
    def get_tokenizer(self, text):
        x = self.tokenizer(text, padding="max_length", truncation=True)
        for k, v in x.items():
            x[k] = torch.tensor(v)
        return x

In [36]:
dt = ReviewDataset(tokenizer, train_arr, target)
dl = torch.utils.data.DataLoader(dt, batch_size=2, shuffle=False)
batch = next(iter(dl))
batch

{'x': {'input_ids': tensor([[    2,  2059, 18764,  ...,     0,     0,     0],
         [    2,  7640,  8078,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])},
 'y': tensor([[1.],
         [0.]])}

In [37]:
class Net(torch.nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.pre_model = AutoModel.from_pretrained(model_name)
        self.fc_out = torch.nn.Linear( self.pre_model.config.hidden_size, 1)

    def forward(self, x):
        x = self.pre_model(**x)
        # x[0]: 모든 시점의 히든출력 batch, seq, features
        # x[1]: CLS 토큰의 히든출력 batch, features
        return self.fc_out(x[1])

In [38]:
model = Net(model_name)
model(batch["x"])

tensor([[ 0.0271],
        [-0.0913]], grad_fn=<AddmmBackward0>)

In [39]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    epoch_loss = 0
    model.train() # 학습 모드
    for batch in tqdm(dataloader):
        pred = model( batch["x"].to(device) )
        loss = loss_fn( pred, batch["y"].to(device) )

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)
    return epoch_loss

In [40]:
@torch.no_grad()
def test_loop(dataloader, model, loss_fn, device):
    epoch_loss = 0
    pred_list = []
    act_func = torch.nn.Sigmoid()
    model.eval() # 평가 모드
    for batch in tqdm(dataloader):
        pred = model( batch["x"].to(device) )
        if batch.get("y") is not None:
            loss = loss_fn( pred, batch["y"].to(device) )
            epoch_loss += loss.item()

        pred = act_func(pred) # logit 값을 확률로 변환
        pred = pred.to("cpu").numpy() # cpu 이동후 ndarray 로변환
        pred_list.append(pred)

    epoch_loss /= len(dataloader)
    pred = np.concatenate(pred_list)
    return epoch_loss, pred

In [41]:
batch_size = 8 # 배치 사이즈
loss_fn = torch.nn.BCEWithLogitsLoss() # 손실 객체

In [42]:
reset_seeds(42) 

# 학습용 데이터로더 객체
train_dt = ReviewDataset(tokenizer, train_arr, target)
train_dl = torch.utils.data.DataLoader(train_dt, batch_size=batch_size, shuffle=True)

# 모델 객체와 옵티마이저 객체 생성
model = Net(model_name).to(device)
optimizer = torch.optim.Adam( model.parameters(), lr=2e-5 )
loss_fn = torch.nn.BCEWithLogitsLoss()

# epochs 조정
epochs = 4
for epoch in range(epochs):
    train_loop(train_dl, model, loss_fn, optimizer, device)
    
torch.save(model.state_dict(), "quality.pth") # 퀄리티 가중치 저장!

  0%|          | 0/288 [00:00<?, ?it/s]

  0%|          | 0/288 [00:00<?, ?it/s]

  0%|          | 0/288 [00:00<?, ?it/s]

  0%|          | 0/288 [00:00<?, ?it/s]